In [37]:
# import the required libraries
import os
import re
import csv
import pandas as pd
import numpy as np
import networkx as nx

In [38]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = '..//data//'
input_folder = 'input//'
input_pred = folderlink+input_folder+"new_predikanten.csv"

In [39]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [40]:
jaar_vertrek_type = {'jaar vertrek': pd.Int64Dtype(), 'ind_id': pd.Int64Dtype(),'dag intrede': pd.Int64Dtype(), 'dag vertrek': pd.Int64Dtype() }
df = pd.read_csv(input_pred, sep=',', dtype= jaar_vertrek_type, encoding='utf-8')

In [41]:
# The dataset is split in two. Since the dataset is composed of rows that have information about where someone went to ('vertrek naar of vanwege') and where he or she came from ('Herkomst'), which is foremost for ministers from after 1816, and rows that have only a year when they move position a cut was made based on the field Herkomst having values NaN.
# The former is named dm_part2, the latter dm_part1. 

dm_part1 = df[df['Herkomst'].isna()]

In [42]:
dm_part1['predikant'] = df['predikant'].str.replace(' ', '')
dm_part1['pred_year_start'] = dm_part1['predikant'].astype(str)+'_'+dm_part1['jaar intrede'].astype(str)
dm_part1['pred_year_end'] = dm_part1['predikant'].astype(str)+'_'+dm_part1['jaar vertrek'].astype(str)

C:\Windows\Temp\ipykernel_14472\1813838777.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part1['predikant'] = df['predikant'].str.replace(' ', '')
C:\Windows\Temp\ipykernel_14472\1813838777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part1['pred_year_start'] = dm_part1['predikant'].astype(str)+'_'+dm_part1['jaar intrede'].astype(str)
C:\Windows\Temp\ipykernel_14472\1813838777.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [43]:
tdf = dm_part1
variable_end = 'pred_year_end'
variable_start = 'pred_year_start'
variable_name = 'predikant'


In [44]:
def create_node_paths_dataframe(dataframe):
    # Generate the directed graph G
    G = nx.DiGraph()
    for _, row in dataframe.iterrows():
        G.add_edge(row[variable_start], row[variable_end], name=row[variable_name])

    # Find connected components and assign path IDs
    paths = list(nx.connected_components(G.to_undirected()))
    path_ids = list(range(1, len(paths) + 1))
    node_path_pairs = []
    for path_id, path_nodes in zip(path_ids, paths):
        node_path_pairs.extend([(node, path_id) for node in path_nodes])

    # Create a DataFrame from the node-path pairs
    node_paths = pd.DataFrame(node_path_pairs, columns=[variable_start, 'individual'])

    # Join the original DataFrame with the node-paths DataFrame
    joined = pd.merge(dataframe, node_paths, left_on=variable_start, right_on=variable_start, how='left')

    return joined

In [45]:
result_dm_part1 = create_node_paths_dataframe(tdf)

In [46]:
individuals_p1 = result_dm_part1[['pid',variable_name,'individual']].copy()

In [47]:
dm_part1_max = individuals_p1['individual'].max()


In [48]:
dm_part2 = df[df['Herkomst'].notna()]

In [49]:
dm_part2['gemeente'] = dm_part2['gemeente'].str.replace(' ', '')
dm_part2['vertrek naar of vanwege'] = dm_part2['vertrek naar of vanwege'].str.replace(' ', '')

dm_part2['gemeente'] = dm_part2['gemeente'].str[:6]
dm_part2['vertrek naar of vanwege'] = dm_part2['vertrek naar of vanwege'].str[:6]


C:\Windows\Temp\ipykernel_14472\541579337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['gemeente'] = dm_part2['gemeente'].str.replace(' ', '')
C:\Windows\Temp\ipykernel_14472\541579337.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['vertrek naar of vanwege'] = dm_part2['vertrek naar of vanwege'].str.replace(' ', '')
C:\Windows\Temp\ipykernel_14472\541579337.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [50]:
dm_part2['predikant'] = dm_part2['predikant'].str.replace(' ', '')
dm_part2['pred_start'] = dm_part2['predikant'].astype(str)+'_'+dm_part2['gemeente'].astype(str)+'_'+dm_part2['jaar intrede'].astype(str)
dm_part2['pred_end'] = dm_part2['predikant'].astype(str)+'_'+dm_part2['vertrek naar of vanwege'].astype(str)+'_'+dm_part2['jaar vertrek'].astype(str)

C:\Windows\Temp\ipykernel_14472\4125051709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['predikant'] = dm_part2['predikant'].str.replace(' ', '')
C:\Windows\Temp\ipykernel_14472\4125051709.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_part2['pred_start'] = dm_part2['predikant'].astype(str)+'_'+dm_part2['gemeente'].astype(str)+'_'+dm_part2['jaar intrede'].astype(str)
C:\Windows\Temp\ipykernel_14472\4125051709.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [51]:
dm_part2 = dm_part2.sort_values(by='pred_start')

In [52]:
tdf = dm_part2
variable_end = 'pred_end'
variable_start = 'pred_start'
variable_name = 'predikant'


In [53]:
result_dm_part2 = create_node_paths_dataframe(tdf)

In [54]:
individuals_p2 = result_dm_part2[['pid',variable_name,'individual']].copy()

In [56]:
individuals_p2['individual']= individuals_p2['individual'] + dm_part1_max

In [57]:
individuals = pd.concat([individuals_p1, individuals_p2], ignore_index=True)

In [58]:
unique_individuals = individuals.drop_duplicates(subset=['individual'])

In [59]:
unique_individuals.head(50)

,pid,predikant,individual
0,35685,Aalburg;Johannesvan,1
1,22489,Aalst;Corneliusvan,2
2,46953,Aalst;Gerardusvan,3
5,741,Aalst;Wilhelmus,4
6,8276,Aalstius;Henricus,5
7,32102,Aalstius;Johannes,6
11,21435,Aalstius;Johannes,7
14,5357,Aalstius;Leonardus,8
16,4078,Aalstius;Petrus,9
19,3584,Aalstius;Wilhelmus,10


In [60]:
unique_individuals.describe()

,pid,individual
count,31656.000000,31656.000000
mean,24293.779410,15828.500000
std,15503.700562,9138.444397
min,1.000000,1.000000
25%,10421.750000,7914.750000
50%,23030.500000,15828.500000
75%,37313.250000,23742.250000
max,53646.000000,31656.000000
